In [36]:
import pandas as pd
import numpy as np

# Input

In [93]:
df=pd.read_csv("raw/uci-news-aggregator.csv", sep = ",",skipinitialspace=True) 
#skip initial whitespace else Category might not work

In [94]:
df.head()


,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


# data processing

In [95]:
import string
df['CATEGORY'] = df['CATEGORY'].map({ 'b': 1, 't': 2, 'e': 3, 'm': 4 }) 
#b : business -- t : science and technology -- e : entertainment -- m : health
# df['TITLE'] = df.TITLE.map(
#     lambda x: x.lower().translate(str.maketrans('','', string.punctuation)) 
# )
#replace "," to all ASCII punctuation

df.head()
df.groupby('CATEGORY').agg('count')

,ID,TITLE,URL,PUBLISHER,STORY,HOSTNAME,TIMESTAMP
CATEGORY,,,,,,,
1,115967,115967,115967,115966,115967,115967,115967
2,108344,108344,108344,108343,108344,108344,108344
3,152469,152469,152469,152469,152469,152469,152469
4,45639,45639,45639,45639,45639,45639,45639


# train test split

In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['TITLE'], 
    df['CATEGORY'], 
    random_state = 1
)

print("Training dataset: ", X_train.shape[0])
print("Test dataset: ", X_test.shape[0])

Training dataset:  316814
Test dataset:  105605


# feature extraction through bagging of words

In [97]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(stop_words = 'english')
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

# NB for classification

In [98]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [99]:
predictions = naive_bayes.predict(testing_data)
predictions

array([1, 1, 3, ..., 1, 1, 4], dtype=int64)

# Evaluation metrics

In [92]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
#multi class average choose weighted
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Accuracy score:  0.928336726481
Recall score:  0.928336726481
Precision score:  0.928405793865
F1 score:  0.928361147966


In [100]:
count_vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)